In [1]:
from lyricsgenius import Genius
from collections import Counter
import re
from nltk.tokenize import TreebankWordTokenizer
import string
import pandas as pd
import requests
from tqdm.notebook import tqdm
import pickle

path = r'C:\Users\chris\Documents\GitHub\cs4300sp2021-rad338-jsh328-rpp62-cmc447\sample_data'

tokenizer = TreebankWordTokenizer()
punct = set(string.punctuation)
punct.update({"''", "``", ""})
genius = Genius('bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p')
genius.verbose = False

In [11]:
df1 = pd.read_csv(path + r"\SpotifyAudioFeaturesApril2019.csv")
df1 = df1.loc[df1.instrumentalness < 0.5].reset_index(drop = True) # drop songs that are mostly instrumental

In [12]:
len(df1)

99036

In [19]:
df1_sample = df1.sample(n = 5000, random_state = 1)

In [20]:
df1_sample

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
62741,Rikko Dunez,35x6RlfDH87X4u1cgIKubI,Talkin',0.334000,0.563,232699,0.5270,0.000000,4,0.141,-9.419,0,0.2810,129.861,4,0.147,0
35566,Jim Rystron,4Qg4a6CbWV7U0GIqzkeqId,Walkin' in the Parlor,0.983000,0.594,62798,0.0286,0.000950,2,0.200,-21.146,1,0.0815,98.037,4,0.977,2
30314,D. LYNCH,5m6rUoikFmlviz0u1kZ9UH,Don't Breathe,0.113000,0.881,254473,0.9260,0.000000,0,0.115,-6.086,0,0.2470,99.953,4,0.354,24
32055,Beatrice Markus,6uWcz9DKYqaGuANysINgc1,Burstin' Out,0.007120,0.580,189092,0.8670,0.004290,1,0.333,-5.950,0,0.0555,146.028,4,0.358,1
18932,POLIÇA,0UkB0ISxcNoOnEh5sUMD1g,Agree,0.736000,0.586,201747,0.6210,0.000382,9,0.150,-8.082,0,0.0307,102.989,4,0.456,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21968,Taysav,7EQtd9bbrrhnjFfabqxge9,Once Upon A Time,0.342000,0.630,186514,0.8320,0.000000,7,0.325,-4.292,0,0.2920,165.408,4,0.735,21
14431,The Lonely Biscuits,1JiPojrxU3xlHdUosquh1L,Talk About,0.118000,0.683,201613,0.5800,0.000000,11,0.119,-7.945,1,0.0472,128.032,4,0.687,34
32211,June,5NDIseC2L12LTHoO7Zl328,Interlude (feat. Trae Tha Truth),0.965000,0.566,44319,0.8750,0.251000,0,0.239,-7.648,1,0.1500,84.913,1,0.618,14
94808,Jeremy Zucker,3PMsgFdVtCeTgjzE8X9TTs,better off - filous remix,0.164000,0.782,176347,0.5640,0.006890,8,0.182,-7.752,0,0.2260,101.058,4,0.448,69


In [28]:
df1_sample.to_csv(path + r"\sample5000_SAFApril2019.csv", index = False)

In [2]:
# df = pd.read_csv(path + r"\sample_scraped_data200.csv")

In [21]:
def match(a, b):
    return (a in b) or (b in a)

def get_lyrics(song_name, song_artist):
    while True:
        try:
            s = genius.search_song(song_name, song_artist, get_full_info = False)
            break
        except:
            pass
    if s:
        s = s.to_dict()
        query_name, query_artist = song_name.lower(), song_artist.lower()
        returned_name, returned_artist = s['title'].lower(), s['artist'].lower()

        if match(query_artist, returned_artist):
            return s['lyrics'].lower()

In [24]:
def get_all_lyrics(df, uri_colname = 'uri', trackname_colname = 'name', artist_colname = 'artist'):
    """
    df: DataFrame; uri, name, and artist must be columns in the dataframe
    
    returns a dictionary where key = song uri and value = Counter of tokenized lyrics
    """
    cnt_dict = dict() #dict(uri: Counter(lyric tokens))
    pbar = tqdm(total=len(df))
    for _, row in df.iterrows():
        uri, name, artists = row[[uri_colname, trackname_colname, artist_colname]]
        
        
        for s in ["-", "(", "feat."]:
            ix = name.find(s)
            if s != -1:
                name = name[:ix].strip()
            
        artist = artists.split(",")[0]
        lyrics = get_lyrics(name, artist)
        if lyrics:
            lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
            tokens = [t for t in tokenizer.tokenize(lyrics) if t not in punct]
            cnt = Counter(tokens)
            cnt_dict[uri] = cnt
        pbar.update(1)
    return cnt_dict

In [25]:
lyrics_dict1 = get_all_lyrics(df1_sample, 'track_id', 'track_name', 'artist_name')
with open(path + r'\sample5000_SAFApril2019_lyrics.pkl', 'wb') as f:
    pickle.dump(lyrics_dict1, f)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [5]:
# lyrics_dict = get_all_lyrics(df)
# with open(path + r'\sample_scraped_data200_lyrics.pkl', 'wb') as f:
#     pickle.dump(lyrics_dict, f)

  0%|          | 0/1066 [00:00<?, ?it/s]